To run this file we have to use the files baseline.pt and cnn.pt in the submitted files. Upload them in google collab and run the code block in order.

In [1]:
import torch.nn as nn

In [2]:
!pip3 install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 5.1 MB/s 
     |████████████████████████████████| 270 kB 58.0 MB/s 
     |████████████████████████████████| 112 kB 55.5 MB/s 
     |████████████████████████████████| 2.3 MB 57.5 MB/s 
     |████████████████████████████████| 84 kB 3.0 MB/s 
     |████████████████████████████████| 84 kB 2.7 MB/s 
     |████████████████████████████████| 57 kB 4.4 MB/s 
     |████████████████████████████████| 212 kB 61.1 MB/s 
     |████████████████████████████████| 55 kB 3.3 MB/s 
     |████████████████████████████████| 54 kB 3.3 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 80 kB 7.9 MB/s 
     |████████████████████████████████| 68 kB 7.1 MB/s 
     |████████████████████████████████| 46 kB 3.8 MB/s 
     |████████████████████████████████| 856 kB 43.6 MB/s 
     |████████████████████████████████| 4.1 MB 38.

In [3]:
class Baseline(nn.Module):
  def __init__(self, embedding_dim, vocab):
    super(Baseline, self).__init__()
    self.embedding = nn.Embedding.from_pretrained(vocab.vectors)
    self.fc = nn.Linear(embedding_dim, 1)

  def forward(self, x, lengths=None):
    #x has shape [sentence length, batch size]
    embedded = self.embedding(x)
    average = embedded.mean(0) # [sent len, batch size, emb dim]
    output = self.fc(average)
    output = torch.sigmoid(output)
    return output

In [7]:
class CNN(nn.Module):
    def __init__(self, embedding_dim, vocab):
        super(CNN, self).__init__()
        self.embed_layer = nn.Embedding.from_pretrained(vocab.vectors)
        self.conv1 = nn.Conv2d(1, 50, (2, 100), 1, bias=False)
        self.conv2 = nn.Conv2d(1, 50, (4, 100), 1, bias=False)
        self.activation = nn.ReLU()
        self.fc1 = nn.Linear(100, 1)

    def forward(self, x):
        length = len(x)
        embed_result = self.embed_layer(x)
        #print(embed_result.shape)
        embed_result = embed_result.permute(1, 0, 2)
        #print(embed_result.shape)
        embed_result = embed_result.unsqueeze(1)
        #print(embed_result.shape)

        cnn1 = self.conv1(embed_result)
        #print(cnn1.shape)
        cnn2 = self.conv2(embed_result)
        #print(cnn2.shape)

        cnn1_activation = self.activation(cnn1)
        #print(cnn1_activation.shape)
        cnn1_activation = cnn1_activation.squeeze()
        #print(cnn1_activation.shape)
        cnn2_activation = self.activation(cnn2)
        #print(cnn2_activation.shape)
        cnn2_activation = cnn2_activation.squeeze()
        #print(cnn2_activation.shape)

        cnn1_maxpool = nn.functional.max_pool1d(cnn1_activation, int(length - 1))
        #print(cnn1_maxpool.shape)
        cnn2_maxpool = nn.functional.max_pool1d(cnn2_activation, int(length - 3))
        #print(cnn2_maxpool.shape)

        cnn1_result = cnn1_maxpool.squeeze()
        #print(cnn1_result.shape)
        cnn2_result = cnn2_maxpool.squeeze()
        #print(cnn2_result.shape)

        concat_result = torch.cat((cnn1_result, cnn2_result))
        #print(concat_result.shape)
        prediction = self.fc1(concat_result)
        prediction = torch.sigmoid(prediction)
        #print(prediction.shape)
        return prediction

In [10]:
import gradio as gr
import torch
import torchtext

def classifyBaseline(text):
  glove = torchtext.vocab.GloVe(name="6B", dim=100)
  baseLineModel = torch.load("baseline.pt")

  cNNModel = torch.load("cnn.pt")

  tokens = text.split()
  token_ints = [glove.stoi.get(tok, len(glove.stoi)-1) for tok in tokens]
  token_tensor = torch.LongTensor(token_ints).view(-1,1)
  baselinePrediction = baseLineModel(token_tensor)

  cNNPrediction = cNNModel(token_tensor)

  baselineProb = int(baselinePrediction.detach().numpy()*1000)/1000
  cNNProb = int(cNNPrediction.detach().numpy()*1000)/1000

  if baselineProb > 0.5:
        a,b = "subjective", baselineProb
  else:
       a,b = "objective", baselineProb

  if cNNProb > 0.5:
        c,d = "subjective", cNNProb
  else:
       c,d = "objective", cNNProb
  return a,b,c,d

output1 = gr.Textbox(label="Baseline classification")
output2 = gr.Textbox(label="Baseline prediction probability")

output3 = gr.Textbox(label="CNN classification")
output4 = gr.Textbox(label="CNN prediction probability")

demo1 = gr.Interface(fn=classifyBaseline, inputs="text", outputs=[output1, output2, output3, output4])
demo1.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://14274.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x7f9454529050>,
 'http://127.0.0.1:7860/',
 'https://14274.gradio.app')